In [249]:
import sys
from itertools import product, chain
from collections import defaultdict

from datasets import Dataset

sys.path.insert(0, '/home/ogalolu/thesis/pre-training-multilingual-document-encoders/clef')
from evaluate import  _get_rerank_dir
from clef_dataloaders.clef_dataloader import load_clef_rerank, load_relevance_assessments, load_clef

# Config

In [284]:
qlang = "en"
dlang = "en"
year = "2002"

In [285]:
doc_ids, documents, query_ids, queries, relass = load_clef(qlang, dlang, year)

# Initial Checks

In [286]:
s1 = set(relass.keys())
s2 = set(query_ids)
print("There are no relevance assesment for the following queries:",s2 - s1)

There are no relevance assesment for the following queries: {96, 132, 101, 110, 117, 118, 93, 127}


In [287]:
ids = list(chain.from_iterable(list(relass.values())))
print("Number of available query-document pairs:", len(ids))

Number of available query-document pairs: 821


# Obtain Training Instances

In [288]:
instance_list = list()
for q_ind, d_ind in relass.items():
    temp_query = [queries[query_ids.index(q_ind)]]
    temp_documents = [documents[doc_ids.index(idx)] for idx in d_ind]
    temp_list = list(product(temp_query, temp_documents))
    instance_list.extend(temp_list)

assert len(instance_list) == len(ids)

In [289]:
instance_dic = defaultdict(list)
for q, d in instance_list:
    instance_dic["query"].append(q)
    instance_dic["document"].append(d)

# Create Dataset

In [290]:
dataset = Dataset.from_dict(instance_dic)
print(dataset[:10])

{'query': [' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' AI in Latin America   Amnesty International reports on human rights in Latin America. ', ' U.N. sanctions against Iraq   What measures has Iraq taken to effect the lifting of\r\nthe U.N. economic embargo and political sanctions imposed after\r\nits invasion of Kuwait in 1990? ', ' U.N. sanctions against Iraq   What measures has Iraq taken to effect the lifting of\r\nthe U.N. economic embargo and political sanctions imposed after\r\nits i

In [291]:
dataset = dataset.train_test_split(test_size=0.1)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['query', 'document'],
        num_rows: 738
    })
    test: Dataset({
        features: ['query', 'document'],
        num_rows: 83
    })
})


In [293]:
root_dir = "/work/ogalolu/datasets"
path = os.path.join(root_dir, "clef_2002")
dataset.save_to_disk(path)